In [2]:
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import settings
import time

def db_connect():
    """
    Performs database connection using database settings from settings.py.
    Returns sqlalchemy engine instance
    """
    return create_engine(URL(**settings.DATABASE))

db = db_connect()

In [155]:
#read in list of icustay_ids
df = pd.read_csv('icustay_list.csv', names = ['icustay_id'])
ids = list(df.icustay_id.values)

In [ ]:
#populate new w_tables
populate_wtables(ids, chart_tables, db)

In [ ]:
#build new csvs with all itemids and correct icustay_ids
get_itemid_sum(w_chart_tables, db)

In [ ]:
#make df out of all w_chartevent csv files
import glob
filenames=glob.glob('w_chartevents_*')
dflist=[]
for filename in filenames:
    df = pd.read_csv(filename, index_col = 0)
    dflist.append(df)

chart_df = pd.concat(dflist)

In [123]:
#pull the d_items database (prepping to join to chart_df)
que = '''select * from d_items'''
d_items_df = pd.read_sql(que, db)

#joining the df
feat_df = chart_df.merge(d_items_df[['itemid', 'label']], on='itemid')

#making the ratio 
feat_df['ratio'] = feat_df.num_patients/18000

feat_df.sort_values(by='num_patients', ascending = False, inplace=True)

In [341]:
#comparing itemids in feat_df vs. itemids in keep list 

keep_items = pd.read_csv('Keep_items.txt', names = ['itemsid'])
keep_set = set(keep_items.itemsid.values)
all_set = set(feat_df.itemid.values)
cut_set = all_set.difference(keep_set)
len(cut_set)

3167

In [97]:
#query itemid + distinct number of patients that itemid has been applied to
#save the query result in a csv file 
import time
#time.sleep(60*60) this was to give sam's query an hour to run
def get_itemid_sum(chart_tables, db):
    import time
    for chart in chart_tables:
        filename = chart + '.csv'
        start = time.time()
        que = '''select itemid, count(distinct subject_id) as num_patients
                from %s 
                group by itemid ''' % chart
        df = pd.read_sql(que, db)
        df.to_csv(filename)
        print(chart)
        print(time.time()-start)
        df = None

In [ ]:
def populate_wtables(ids, ref_tables, db):
    from sqlalchemy import text
    import time
    #first, check to make sure all tables are empty. if not, break!
    for table in ref_tables:
        w_table = "w_" + table
        #print(w_table)
        que = '''select * from %s limit 5''' % w_table
        df = pd.read_sql(que, db)
        assert(len(df) < 1), "Table %s is not empty!" % w_table
    

    #okay, tables are empty and ready for population!
    #loop through every table
        #in each table, copy ids from 
    for table in ref_tables:
        start_time = time.time()
        for icu_id in ids:
            w_table = "w_" + table
            command =  '''INSERT INTO %s select * from %s where icustay_id = %d'''% (w_table, table, icu_id)
            sql = text(command)
            db.execute(sql)
        print(table)
        print(time.time()-start_time)
'''
#### Function 1 - Create new tables, scraping ICU IDs

idea:
    given ICUSTAYIDs, POPULATE new chartevent tables ONLY with those (scrape out babies and dead people)
    CHECK IF DB EXISTS, IF SO BREAK

input:
    ICUSTAYIDs, db, chartables
    (db to make sure we dont fuck this up)

do:
    for table in chart_tables:
        for id in ICUSTAYIDs:
            insert into w_chartevents_1 select * from chartevents_1 where icustay_id = 2873

output:
    returns nothing, just runs sql code
'''

In [ ]:
'''
##### Function 1.5 - Create new tables 
idea:
    create new tables, chartevents_1-14, using same schema. DO NOT POPULATE, ONLY CREATE
input:
    list of old table names
    db
do:
    for table in table names:
        create new table w/ same schema, name is changed to w_(oldname)
'''

def spawn_wtables(old_tables, db):
    from sqlalchemy import text
    for table in old_tables:
        new_table = 'w_' + table
        command = '''create table %s as select * from %s where 1=0''' % (new_table, table)
        sql = text(command)
        db.execute(sql)

In [99]:
#build new SQL tables

#get list of all tables in db
from sqlalchemy import MetaData
start_time = time.time()
m = MetaData(bind=db)
m.reflect()
tables = list(m.tables.keys())
time.time()-start_time

# from list of tables, get all chartevent tables
import re
tab_re = re.compile(r'^(w_chartevents_.*)')
chart_tables = []
for table in tables:
    name=re.findall(tab_re, table)
    if name:
        chart_tables.append(name[0])
w_chart_tables = sorted(chart_tables)

['w_chartevents_1',
 'w_chartevents_10',
 'w_chartevents_11',
 'w_chartevents_12',
 'w_chartevents_13',
 'w_chartevents_14',
 'w_chartevents_2',
 'w_chartevents_3',
 'w_chartevents_4',
 'w_chartevents_5',
 'w_chartevents_6',
 'w_chartevents_7',
 'w_chartevents_8',
 'w_chartevents_9']

In [ ]:
cut_set

In [ ]:
items_dict = bin_itemids(ids, 'w_chartevents_')
items_dict

In [ ]:
del_itemids(list(cut_set), 'w_chartevents_', db)

w_chartevents_10
1.7301506996154785
w_chartevents_2
53.15837121009827
w_chartevents_5
187.84087920188904
w_chartevents_9
0.028972387313842773


In [353]:
def del_itemids(ids, base_name, db):
    import time
    '''Takes list of itemids, base chart name, and db. Deletes all itemids from respective tables in db.'''
    items_dict = bin_itemids(ids, base_name)
    for table, ids in items_dict.items():
        start_time=time.time()
        for item_id in ids:
            command =  '''DELETE FROM %s WHERE itemid = %d'''% (table, item_id)
            sql = text(command)
            db.execute(sql)
        print(table)
        print(time.time()-start_time)
        

In [346]:
def bin_itemids(ids, base_name):
    '''Takes a list of itemids, bin itemids into their respective chart based on base_name, output dictionary'''
    items_dict = {}
    items_index = [210,250,614,640,742,1800,2700,3700,4700,6000,7000,8000,220074,323769]
    for index, item_bin in enumerate(items_index):
        chart = base_name + str(index+1)
        binned_ids = [i for i in ids if i < item_bin]
        items_dict[chart] = binned_ids
        ids = [i for i in ids if i not in binned_ids]
    return items_dict
        

In [ ]:
#check amount of unique patients w_chartevents 
dflist=[]
for table in w_chart_tables:
    start_time=time.time()
    command =  '''select distinct subject_id from %s''' % table
    sql = text(command)
    dflist.append(pd.read_sql(sql, db))
    print(time.time()-start_time)
df=pd.concat(dflist)

In [ ]:
def feature_builder():
    '''Takes list of itemids
        bucket itemids
        create new table with icustay + itemids as columns
        
        build data one patient at a time
        for 
        for chart, itemids in itembuck:    
            for item in itemids:
                get all measurements + dates
                put into dictionary
            put dictionary into df
        return df
    '''

In [275]:
def drop_wchart_columns(tables, columns):
    '''takes list of columns and drops them from every table in tables'''
    for table in tables:
        start_time = time.time()
        for column in columns:
            command =  '''ALTER TABLE %s DROP %s'''% (table, column)
            sql = text(command)
            db.execute(sql)
        print(time.time()-start_time)

In [273]:
columns = ['hadm_id',
'cgid',
'warning',
'stopped',
'resultstatus',
'error']

['hadm_id', 'cgid', 'warning', 'stopped', 'resultstatus', 'error']

In [ ]:
def get_wchart_data(tables, db):
    '''Gets all tables in list from sql server'''
    dflist = []
    for table in tables:
        start_time = time.time()
        command =  '''select * from %s''' % table
        sql = text(command)
        dflist.append(pd.read_sql(sql, db))
        print(table)
        print(time.time()-start_time)
    df = pd.concat(dflist)
    print('Writing to pickle...')
    df.to_pickle('w_chartevents.p')
    dflist = None
    return df

## Everything below here is purely testing/development, should be taken as bad code

In [ ]:
def flatten_wchart(df):
'''
after getting all w_charts into pandas:
build features!
'''
    superdict = {}
    for icustay_id in df.icustay_id.unique():
        for itemid in df.itemid.unique():
            create dict w/ every item
            superdict['patient'].append(item)
           
    features_df = pd.dataframe(superdict)


In [268]:
asdict = {'icuid':123}
a = asdf[['charttime', 'valuenum', 'valueuom']].values.tolist()
asdict['itemid213'] = [a]
asdf2 = pd.DataFrame(asdict)
asdf2


,icuid,itemid213
0,123,"[[2101-10-21 04:00:00, 3.0, points], [2101-10-..."


In [321]:
superdict={}
superdict['icuid'] = 'asd'
superdict['item1']  = [[[1, 2, 3], [1, 2,3], [1, 2, 3], [1, 2, 3]]]
superdict['item3']  = [[[1, 2, 3], [1, 2,3], [1, 2, 3], [1, 2, 3]]]

asdf2 = pd.DataFrame(superdict)
asdf2

,icuid,item1,item3
0,asd,"[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]","[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]"


In [322]:
asdf2.to_csv('test.csv')
asdf3 = pd.to_pickle('test.csv')
asdf3

,Unnamed: 0,icuid,item1,item3
0,0,asd,"[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]","[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]"


In [320]:
def get_most_recent(entries):
    '''Sort by time, return most recent data '''
    sorted_entries = sorted(entries, key = lambda x: x[0])
    return sorted_entries[-1][1]

get_most_recent(entries)
asdf2['item1'] = asdf2['item1'].apply(item_map['item1'])
asdf2

,icuid,item1,item3
0,asd,2,"[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]"


In [312]:
item_map = {'item1' : get_most_recent}
item_map['item1']

<function __main__.get_most_recent>

In [296]:
entries = asdf[['charttime', 'valuenum', 'valuenum']].values.tolist()
sorted(entries, key = lambda x: x[0])[-1][1]

15.0

In [ ]:
asdf[[['icustay_id']==211552]